In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, pipeline, AutoTokenizer
from IPython.display import display, Markdown
import re
import os

In [14]:
#Import the text_generated
#folder_path = 'processed_data'
#file_path = os.path.join(folder_path, 'pdf2_text.txt')
#file_path = os.path.join(folder_path, 'pdf3_text.txt')
#file_path = os.path.join(folder_path, 'video_text.txt')
#file_path = os.path.join(folder_path, 'scrapping_text.txt')

with open(file_path, 'r', encoding='utf-8') as file:
    full_text = file.read()

In [4]:
#Print function
def show_full_text(text):
    display(Markdown(text))

MODELO T5 BASE
(problema porque me corta las oraciones - truncado)

In [18]:
class T5Summarizer:
    def __init__(self, model_name='t5-base', max_chunk_length=512):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.max_chunk_length = max_chunk_length

    def chunk_text(self, text):
        # Tokenizar el texto y dividirlo en chunks según el máximo de tokens permitidos
        tokens = self.tokenizer.encode(text, return_tensors="pt", truncation=False)
        chunk_size = self.max_chunk_length
        
        # Dividir los tokens en chunks
        chunks = [tokens[0][i:i + chunk_size] for i in range(0, tokens.shape[1], chunk_size)]
        return chunks

    def summarize_chunk(self, chunk, max_length=256, num_beams=5, length_penalty=2.0, no_repeat_ngram_size=3):
        # Generar el resumen de un chunk
        summary_ids = self.model.generate(
            chunk.unsqueeze(0),
            max_length=max_length,
            num_beams=num_beams,
            length_penalty=length_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=True
        )
        # Decodificar el resumen
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    def summarize(self, text, max_length=256, num_beams=5, length_penalty=2.0, no_repeat_ngram_size=3):
        # Dividir el texto en chunks
        chunks = self.chunk_text(text)
        
        # Resumir cada chunk y concatenar los resultados
        summaries = [self.summarize_chunk(chunk, max_length, num_beams, length_penalty, no_repeat_ngram_size) for chunk in chunks]
        
        # Unir los resúmenes de los chunks
        return " ".join(summaries)
    

# Ejemplo de uso:
summarizer = T5Summarizer()
summary = summarizer.summarize(full_text)

show_full_text(summary)

c:\Users\Agustina Torres\Documents\DEV\IMF\TFM\SELENIUM\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


en Europao.4(4):263-264 263 71. doi:10.1016/j.4(4)4(4)4(4):263- 71  del abuso y depen- dencia de sustancias en todos los pases europeos... - -. -. -4:2 2 , y los trabajos sobre la psicopatologa comórbida, con influencias socioculturales, incluyendo los diferentes contextos legales. Esto nos permitirá ayudar a la integración de la Unión Europea, por ejemplo, entre las cien- tficas de áreas básicas el Colegio Europeo de Ciencias de las Adic- tions. Ayudara a su vez a crear un marco de trabajo ade- cuado que facilita la investigación multinacional. A. Invitamos a todos los cientficos que tienen en las áreas relacionadas con los trastornos adictivos. a la práctica. — • Facilitación de la transferencia de la evidencia existente. B. Mejorar los servicios terapéuticos para los tras- tornos adictivos en Europa: — Promover la investigación sobre el coste-effecti- vidad de las intervenciones. . — Promoverá la investigación para que los pro- gramas y servicios de tratamiento dispongan de sistemas de evaluación, de procesos, y resulta- dos, que permitan determinar la efectividad y eficiencia de las intervenciones que desarrollen. (United Kingdom) Autores Sociedad Espaola de Toxicomanas C/ San Vicente, 112-Pta. 2 46007 Valencia Tel. : 96 352 15 48 e-mail: valderra@setox.org http://www.setoxx.org www.essa.org/ apoyar y esti- mular esta discusión. Especialmente en el orgorgorgorgorgorgorgorgorgorgorgorg.org.orgorgorgorgorgorgorgorgorgorgorgorg or orgorgorgorgorgorgorgorgorgorgorgorgorgorgorgorgorgorgorgorgorgorg/orgorgorgorgorgorgorgorgorgorgorgorgorgorg is a non-profit organization that aims to provide a safe and secure environment for

MODELO T5 BASE PERO CON AJUSTES EN LA DIVISION DE CHUNKS, POR ORACIONES.
(problemas con los videos sin puntos)

In [19]:
class T5Summarizer:
    def __init__(self, model_name='t5-base', max_chunk_length=512):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.max_chunk_length = max_chunk_length

    def split_text_into_chunks(self, text):
        # Dividir el texto en oraciones
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
        
        current_chunk = ""
        chunks = []

        for sentence in sentences:
            # Añadir la oración al chunk actual
            current_chunk += sentence + " "
            tokenized_chunk = self.tokenizer.encode(current_chunk, return_tensors="pt", truncation=False)

            # Si el chunk tokenizado excede el tamaño máximo permitido, guardarlo y empezar uno nuevo
            if tokenized_chunk.shape[1] > self.max_chunk_length:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + " "  # Iniciar nuevo chunk con la oración actual

        # Añadir el último chunk si no está vacío
        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

    def summarize_chunk(self, chunk, max_length=400, min_length=150, num_beams=4, length_penalty=1.2, no_repeat_ngram_size=2):
        inputs = self.tokenizer.encode("summarize: " + chunk, return_tensors="pt", max_length=self.max_chunk_length, truncation=True)
        summary_ids = self.model.generate(
            inputs,
            max_length=max_length,
            min_length=min_length,
            num_beams=num_beams,
            length_penalty=length_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=True
        )
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    def summarize(self, text, max_length=400, min_length=150, num_beams=4, length_penalty=1.2, no_repeat_ngram_size=2):
        # Dividir el texto en chunks respetando oraciones
        chunks = self.split_text_into_chunks(text)
        
        # Resumir cada chunk
        summaries = [self.summarize_chunk(chunk, max_length, min_length, num_beams, length_penalty, no_repeat_ngram_size) for chunk in chunks]
        
        # Post-procesamiento: unir los resúmenes con transiciones suaves
        full_summary = " ".join(summaries)
        return full_summary

# Uso del modelo
summarizer = T5Summarizer()

# Resumir el texto
summary = summarizer.summarize(full_text)


show_full_text(summary)


la prevalencia del abuso y depen- dencia ofrecen sustancias en todos los pases europeo- sade. la investigación de los trastornos - no están suficientemente integrados dentro de la práctica clnica e las decisi- nos polticas. por otra parte se estan produciendo importante n   ­ gra  re- de, l--. h w  t enfoque global incluira, por ejemplo, los nuevos tra- bajos de investigación de genética molecular. y la interacción entre estos factores individuale- s con influencias socioculturale. no reem- plazar o duplicar su trabajo, teniendo a ampliar la Unión Europea; entre la epidemiol n    ­ gra - de-- de- in h i  de de " versiune , la necesidad de un «Colegio Europeo de Ciencias de las Adicciones» («European College on Addiction Sciences») para establecer un foro que facilite y potencie el diálogo con otras organizacio- nes semblables tanto en Europa como fuera de ella. ayudara al cientficos que trabajan entre la versiune  ­ n gra­­-­­n­s - - ( h l  , el Colegio Européen de Trastornos Adictivos celebra los próximos 27 a 29 de marzo de 2003 en Alicante (Espaa). entre los objeti- vo- siguientes: A. Mejorar la contribución de la investigación. B. Promover un abordaje basado à la videncia de los intervenciones, mediante las : — Identificación.  ­­ n­-­ de  h - (  t,  s-- de--. c. Mejorar los servicios terapéuticos para los tras- tornos adictivos en Europa. d. Promoverá más investigación para poder pro- porcionar nueva opciones áreas de evaluación, de proceso y resulta- dos, que permitan determinar la efficiencia de las intervenciones que desarrollen. la capacidad de electra de los paciente  - gra ­ n l   versiune  h, s- de de de in  de el i Symposium Europeo sobre Proble- mas Clnicos relacionados con los Trastornos Adicti- vos desean expresar su agradecimiento al Plan de Ga-licia. la sociedad espaola de Toxicomanas por apoyar y esti-mular esta discusión. los participantes reunidos en Santiago de Compostela, gra   ­­ n­-­n  versiune - de  o l--. h t s marc Auriacombe (France), Manuel Arajo (Spain) and indalecio Carrera (Ireland) e-mail: valderra@setox.org 96 352 15 48. cda: http://www.cochrane drugs and alcohol review group; swiss: www.icromaremmani (italy and EuroPAT); márquez, lvarez,s n - ­­ l   re gra versiune,--..-.s-s (

MODELO MT5 - MEJOR Y MULTILINGUAL
(ANDA PESIMO!!!)

In [20]:
class T5Summarizer:
    def __init__(self, model_name='google/mt5-base', max_chunk_length=512):
        self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
        self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
        self.max_chunk_length = max_chunk_length
    
    def split_text_into_chunks(self, text):
        # Dividir el texto en oraciones
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
        
        current_chunk = ""
        chunks = []

        for sentence in sentences:
            # Añadir la oración al chunk actual
            current_chunk += sentence + " "
            tokenized_chunk = self.tokenizer.encode(current_chunk, return_tensors="pt", truncation=False)

            # Si el chunk tokenizado excede el tamaño máximo permitido, guardarlo y empezar uno nuevo
            if tokenized_chunk.shape[1] > self.max_chunk_length:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + " "  # Iniciar nuevo chunk con la oración actual

        # Añadir el último chunk si no está vacío
        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

    def summarize_chunk(self, chunk, max_length=300, min_length=100, num_beams=6, length_penalty=2.0, no_repeat_ngram_size=3):
        # Agregar el prefijo "summarize: " antes del chunk
        inputs = self.tokenizer.encode("summarize: " + chunk, return_tensors="pt", max_length=self.max_chunk_length, truncation=True)
        summary_ids = self.model.generate(
            inputs,
            max_length=max_length,
            min_length=min_length,
            num_beams=num_beams,
            length_penalty=length_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=True
        )
        
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary


    def summarize(self, text, max_length=300, min_length=100, num_beams=6, length_penalty=2.0, no_repeat_ngram_size=3):
        # Dividir el texto en chunks respetando oraciones
        chunks = self.split_text_into_chunks(text)
        
        # Resumir cada chunk
        summaries = [self.summarize_chunk(chunk, max_length, min_length, num_beams, length_penalty, no_repeat_ngram_size) for chunk in chunks]
        
        # Post-procesamiento: unir los resúmenes con transiciones suaves
        full_summary = " ".join(summaries)
        return full_summary

# Uso del modelo
summarizer = T5Summarizer()

# Resumir el texto
summary = summarizer.summarize(full_text)
#print(summary)


show_full_text(summary)

<extra_id_0> la investigación de los trastornos adictivos, <extra_id_1> en el contexto europeo, y en el desarrollo de la sociedad en todo el mundo. P.P.. <extra_id_2>,.. <extra_id_3>.. <extra_id_12>.... <extra_id_4>.. <extra_id_5>. <extra_id_6>. <extra_id_7>. <extra_id_8>. <extra_id_9>. <extra_id_10>. <extra_id_11> Trastornos Adictivos en Europa <extra_id_12>.. <extra_id_13>. <extra_id_14>. <extra_id_15>.. <extra_id_16>.. <extra_id_17>. Trastorno adictivo en Europa Trastorno <extra_id_18> s <extra_id_0> a la investigación y la comunicación: <extra_id_1> de acuerdo a los objetivos del Colegio Europeo de Ciencias de las Adic- ciones <extra_id_2>.. <extra_id_3>.. <extra_id_4>.. <extra_id_5>.. <extra_id_6>.. a <extra_id_7>.. <extra_id_8>.. <extra_id_9>.  <extra_id_10>. e <extra_id_11>. <extra_id_12>. Adicciones C <extra_id_13>. <extra_id_14>. <extra_id_15>. Adic <extra_id_16>  <extra_id_54>...... <extra_id_18>... <extra_id_18>.. <extra_id_18> de adicciones.. <extra_id_0> de la Sociedad Española de Toxicomanías PSICOLOGÍA <extra_id_1> NOTICIAS Trastornos Adictivos 2002;4(4):263-264 72  <extra_id_2> A <extra_id_3> la investigación y la práctica, <extra_id_4> y  <extra_id_5> a <extra_id_6> a <extra_id_7> a <extra_id_8> a  <extra_id_9> a <extra_id_10> a <extra_id_11> a <extra_id_12> a <extra_id_13>: <extra_id_17> Política y Clínica <extra_id_18>, <extra_id_20>, <extra_id_21> a <extra_id_22> a <extra_id_23> i <extra_id_24>  <extra_id_25>  <extra_id_26>s <extra_id_27>es Pacientes adictivos Trastorno a <extra_id_28>  <extra_id_29>  <extra_id_29> patologías psiquiátricas y somáticas <extra_id_30>es <extra_id_31>es  <extra_id_32>es  <extra_id_33>es  <extra_id_34> i <extra_id_35> <extra_id_0> (Spain) Francisco Javier Auriacombe (Belgium/ <extra_id_1>) José Carlos Pérez de los Cobos (España) Todos los participantes de esta discusión, y  <extra_id_2>..,... <extra_id_3>..., <extra_id_4>..),... <extra_id_5>... <extra_id_6>  <extra_id_13>... <extra_id_14>... <extra_id_15>... <extra_id_16>... <extra_id_17>..  <extra_id_18>.. <extra_id_18>....... <extra_id_19>  <extra_id_20>  <extra_id_21>s <extra_id_22>...,...  <extra_id_23> Alcohol Review Group <extra_id_41> <extra_id_54>...... <extra_id_0> (Spain) José María Valdérraga (Spain), <extra_id_1> (Italy and EuroPAT) Participants: <extra_id_2> (Spain): <extra_id_3> (Spanish): <extra_id_12> <extra_id_53>, <extra_id_56> (Spanish: Sociedad Española de Toxicomanías C/ San Vicente) 112-Pta. 2 46007 Valencia Tel: 96 352 15 48 e-mail: valderra@setox.org http://www.setoxgroup.org/en/ -... -.... -.. )

MODELO PEGASUS (INGLES)

In [4]:
#google/pegasus-xsum
#google/pegasus-large
#oogle/pegasus-multi_news

class PegasusSummarizer:
    def __init__(self, model_name='google/pegasus-multi_news', max_chunk_length=512):
        self.tokenizer = PegasusTokenizer.from_pretrained(model_name)
        self.model = PegasusForConditionalGeneration.from_pretrained(model_name)  # Sin from_tf
        self.max_chunk_length = max_chunk_length

    def split_text_into_chunks(self, text):
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
        
        current_chunk = ""
        chunks = []
        current_length = 0

        for sentence in sentences:
            tokenized_sentence = self.tokenizer.encode(sentence, return_tensors="pt", truncation=False)
            current_length += len(tokenized_sentence[0])

            if current_length > self.max_chunk_length:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + " "
                current_length = len(tokenized_sentence[0])
            else:
                current_chunk += sentence + " "

        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks


    def summarize_chunk(self, chunk, max_length=60, min_length=30, num_beams=4):
        inputs = self.tokenizer(chunk, return_tensors="pt", truncation=True, padding="longest", max_length=self.max_chunk_length)
        summary_ids = self.model.generate(
            inputs['input_ids'],
            max_length=max_length,
            min_length=min_length,
            num_beams=num_beams,
            early_stopping=True
        )
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    


    def summarize(self, text, max_length=60, min_length=30, num_beams=4):
        chunks = self.split_text_into_chunks(text)
        summaries = [self.summarize_chunk(chunk, max_length, min_length, num_beams) for chunk in chunks]
        full_summary = " ".join(summaries)
        return full_summary

# Uso del modelo PEGASUS
pegasus_summarizer = PegasusSummarizer()

# Resumir el texto
summary = pegasus_summarizer.summarize(full_text)
#print(summary)

show_full_text(summary)


c:\Users\Agustina Torres\Documents\DEV\IMF\TFM\SELENIUM\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-multi_news and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


– The European Court of Human Rights has ruled in favor of a Spanish woman who says she was fired from her job as a teacher because she refused to give birth to a child, reports the Telegraph. The court in Strasbourg, France, said that the woman's complaint "did not meet – The European Commission is calling for more research into the effects of alcohol on people's mental health in a bid to cut down on the number of people dying from alcohol-related diseases in Europe, the Telegraph reports. The number of alcohol-related deaths in Europe has more than doubled in – The Colegio Europeo de Ciencias de la Ciencias de las Adicciones, a European College of Addiction Sciences-supported program that helps people struggling with substance abuse, has been forced to close its doors because of a lack of funding, the BBC reports. The program – Spain's government has taken the unusual step of setting up a fund to compensate victims of the country's devastating recession, the Wall Street Journal reports. The 100 million ($140 million) fund will cover victims of the economic crisis who lost their jobs or suffered other hardships, – Spain is in the midst of an economic crisis, and the country's prime minister has called for an end to austerity measures, the BBC reports. Mariano Rajoy made the call at the opening of a two-day conference on economic policy in Santiago de Compostela, Spain, where he – A Spanish court has ordered the release of a man accused of killing his wife's best friend with a hammer in 2013, the AP reports. The court on Tuesday ordered the release of Francisco Javier lvarez Rodrguez after he pleaded not guilty to murder, attempted murder, and

MODELO MBART - MULTILINGUAL

In [12]:
class MultilingualSummarizer:
    source_lang = 'es_XX'  # español
    target_lang = 'es_XX'   # también español

    def __init__(self, model_name='facebook/mbart-large-50-many-to-many-mmt'):
        self.tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
        self.model = MBartForConditionalGeneration.from_pretrained(model_name)

    def summarize(self, text, source_lang = source_lang, target_lang = target_lang, max_length=512, num_beams=5, length_penalty=1.5, no_repeat_ngram_size=2):
        self.tokenizer.src_lang = source_lang
        forced_bos_token_id = self.tokenizer.lang_code_to_id[target_lang]

        inputs = self.tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

        summary_ids = self.model.generate(
            **inputs,
            max_length=max_length,
            forced_bos_token_id=forced_bos_token_id,
            num_beams=num_beams,
            length_penalty=length_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=False
        )

        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

#def preprocess_text(text):
    # Remove irrelevant mentions and other noise
    #clean_text = re.sub(r'\[Música\].*?Compartir', '', text, flags=re.DOTALL).strip()
    # Ensure consistent punctuation and spacing
   #clean_text = re.sub(r'\s+', ' ', clean_text)  # Replace multiple spaces with a single space
    #return clean_text

def split_text(text, max_length):
    sentences = text.split('. ')  # Use period as delimiter
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 > max_length:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += ('. ' if current_chunk else '') + sentence
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

def summarize_text(text, summarizer, max_length=512, chunk_size=1024):
    chunks = split_text(text, chunk_size)
    summaries = []
    for chunk in chunks:
        summary = summarizer.summarize(chunk, max_length=max_length)
        summaries.append(summary)
    return ' '.join(summaries)

# Usage in the code
summarizer = MultilingualSummarizer()
#input_text = preprocess_text(full_text)
summarized_text = summarize_text(full_text, summarizer)

show_full_text(summarized_text)

Adictiva Transtornos 2002;4(4):263-264 263 71 «Documento Galicia sobre Problemas Relacionados con los Adictivos en Europa» There is an increase in the prevalence of abuse and addiction of substances in all European countries, and addictive disorders represent a serious problem for the affected individuals, their families and for society throughout Europe. La investigación de los tras- ctores en el contexto europeo debe conducir a una mejor comprensión del problema subyacente, al desarrollo de tratamientos más eficaces basa- dos en la evidencia científica; y ofrecer un modelo para un mejor integración de las diferentes perspectivas que influyen sobre los diferentes aspectos individuales, de desarrollo, social y cultural en Europa. Este enfoque global incluiría, por ejemplo, los nuevos tra- encos de genes moleculares, neurobiológicos y psychologicos, trabajos sobre la psicopatología combinada, y la interacción de estos factores individuales con influencias socioculturales including los distintos contextos legales. Por el contrario, nuestra intención es añadir una dimensión europea más integrada, teniendo también en cuenta la ampliación de la Unión Europea, y servir como foro que albergue la comunicación y el enriquecimiento mutuo entre cien médicos de áreas básicas y clínicas; entre el epidemio- gía y la salud pública, para ayudar a la planificación de políticas de intervención, lo que nos permitirá ir en contra de las diferencias y las similitudes no sólo entre las drogas, el alcohol e el tabaco, sino también entre los diferentes países europeos, que en su vez facilitaría las colaboraciones de investigación entre grupos de distintos países y permitiría la elaboración de guías e protocolos terapéuticos eficaces y prácticos. En este contexto, el Ministerio de Salud y Bienestar Social, en colaboración con la Oficina del Alto Comisionado de las Naciones Unidas para los Derechos Humanos (ACNUDH), organizó un seminario sobre la salud de los adolescentes en el que participaron representantes de la sociedad civil, los medios de comunicación, las organizaciones no gubernamentales (ONG) y el sector privado, con el fin de examinar las cuestiones relacionadas con los niños y los jóvenes. Envió una carta al Secretario General de las Naciones Unidas sobre el VIH/SIDA, en la que se pedía a la Oficina del Alto Comisionado para los Derechos Humanos (ACNUDH) que prestara asistencia a los gobiernos y a las organizaciones no gubernamentales (ONG) en relación con el problema de la violencia contra la mujer. b) Establezca una red de expertos que podrían ex- plorar las oportunidades de apoyo económico en Europa que faciliten las iniciativas comunes de investigación multi-nacional y interdisciplinar. B. Improve the therapeutic services for las tras- rounds adictivas en Europe: — Tanto en atención primaria como especializada la base de evidencia científica de los enfoques terapéuticos no siempre es suficiente. El Presidente (habla en inglés): De conformidad con el entendimiento alcanzado en las consultas previas, entenderé que la Asamblea General desea aprobar el proyecto de resolución A/C.2/57/L.15/Rev.1, titulado “Convención de las Naciones Unidas contra el Tráfico Ilícito de Estupefacientes y Sustancias Sicotrópicas de 1988”. A/C.2/57/L.18 Temas a) y b) del programa - Informe del Secretario General sobre la situación de los derechos humanos y las libertades fundamentales en la República Democrática del Congo - Proyecto de resolución presentado por el Presidente del Consejo de Seguridad [A C E F I R] En el párrafo 2 de la parte dispositiva del proyecto de presupuesto por programas para el bienio 2006-2007, la Junta recomendó que el Secretario General presentara un informe a la Asamblea General en su quincuagésimo séptimo período de sesiones sobre la situación de los derechos humanos en la República Democrática del Congo.

MODELO BART (INGLES) +  PIPELINES

In [10]:
# Load the translation models and summarization model
translator_es_en = pipeline("translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")
translator_en_es = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Load tokenizer for chunking
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# Function to split the text into smaller chunks using the tokenizer
def chunk_text(text, tokenizer, max_length=512):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    tokens = inputs['input_ids'].squeeze()
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


def summarize_text(text):
    # Translate the text into English
    translated_text_chunks = chunk_text(text, tokenizer)
    translated_text = ""

    for chunk in translated_text_chunks:
        translated_text += translator_es_en(chunk)[0]['translation_text'] + " "

    # Split the translated text into smaller chunks if it's too long
    text_chunks = chunk_text(translated_text.strip(), tokenizer)

    # Generate the summary for each part of the text
    summary = ""
    for chunk in text_chunks:
        input_length = len(chunk.split())
        dynamic_max_length = min(max(30, input_length // 2), 150)  # Ajuste de max_length
        summary_result = summarizer(chunk, max_length=dynamic_max_length, min_length=50, do_sample=False)  # Aumento de min_length
        summary += summary_result[0]['summary_text'] + " "

    # Translate the summary back into Spanish
    final_summary_chunks = chunk_text(summary.strip(), tokenizer)
    final_summary = ""

    for chunk in final_summary_chunks:
        final_summary += translator_en_es(chunk)[0]['translation_text'] + " "

    return final_summary.strip()


# Call the English model
summarized_text = summarize_text(full_text)


show_full_text(summarized_text)


Hay un aumento en la prevalencia del abuso de sustancias y la dependencia en todos los países europeos. Sin embargo, la investigación sigue siendo baja y hay una falta de comunicación entre los grupos de profesionales que trabajan en las diferentes áreas relacionadas.